In [1]:
%matplotlib inline

In [2]:
import os
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

In [3]:
import aimmd
import aimmd.distributed as aimmdd

In [4]:
n_engines = 6

In [5]:
scratch_dir = "/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/"
wdirs = [os.path.join(scratch_dir, f"engine_wdir{i}") for i in range(n_engines)]

In [6]:
[os.mkdir(d) for d in wdirs]

FileExistsError: [Errno 17] File exists: '/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0'

In [7]:
engines = [aimmdd.GmxEngine(gro_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                            top_file=os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                            #mdrun_extra_args="-nt 4",
                            mdrun_extra_args="-ntomp 4",
                            )
           for _ in range(n_engines)]

In [8]:
mdps = [aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp"))
        for _ in range(n_engines)]

In [9]:
mdps[0]

{'title': ['test'], 'cpp': ['/lib/cpp'], 'include': ['-I../top'], 'define': [], 'integrator': ['md-vv'], 'dt': [0.002], 'nsteps': [100000], 'nstxout': [10], 'nstvout': [10], 'nstlog': [10], 'nstenergy': [10], 'nstxout-compressed': [10], 'compressed-x-grps': ['Protein'], 'energygrps': ['Protein', 'SOL'], 'nstlist': [10], 'ns-type': ['grid'], 'cutoff-scheme': ['Verlet'], 'rlist': [1.1], 'coulombtype': ['PME'], 'rcoulomb': [1.1], 'rvdw': [1.1], 'tcoupl': ['Berendsen'], 'tc-grps': ['Protein', 'SOL'], 'tau-t': [0.1, 0.1], 'ref-t': [300.0, 300.0], 'Pcoupl': ['Berendsen'], 'tau-p': [1.0], 'compressibility': [4.5e-05], 'ref-p': [1.0], 'gen-vel': ['no'], 'gen-temp': [300.0], 'gen-seed': [173529], 'constraints': ['all-bonds']}

In [10]:
for mdp in mdps:
    mdp['nstvout'] = 10
    mdp["nstxout"] = 10

In [11]:
mdps[0]

{'title': ['test'], 'cpp': ['/lib/cpp'], 'include': ['-I../top'], 'define': [], 'integrator': ['md-vv'], 'dt': [0.002], 'nsteps': [100000], 'nstxout': [10], 'nstvout': [10], 'nstlog': [10], 'nstenergy': [10], 'nstxout-compressed': [10], 'compressed-x-grps': ['Protein'], 'energygrps': ['Protein', 'SOL'], 'nstlist': [10], 'ns-type': ['grid'], 'cutoff-scheme': ['Verlet'], 'rlist': [1.1], 'coulombtype': ['PME'], 'rcoulomb': [1.1], 'rvdw': [1.1], 'tcoupl': ['Berendsen'], 'tc-grps': ['Protein', 'SOL'], 'tau-t': [0.1, 0.1], 'ref-t': [300.0, 300.0], 'Pcoupl': ['Berendsen'], 'tau-p': [1.0], 'compressibility': [4.5e-05], 'ref-p': [1.0], 'gen-vel': ['no'], 'gen-temp': [300.0], 'gen-seed': [173529], 'constraints': ['all-bonds']}

In [12]:
#for engine, wdir, mdp in zip(engines, wdirs, mdps):
#    engine.prepare(starting_configuration=None, workdir=wdir, deffnm="blub", run_config=mdp)

In [13]:
import time

In [14]:
import mdtraj as mdt
import numpy as np
"""
def psi(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_psi(traj)
    return psi

def alpha_R(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_dihedrals(traj, indices=[[6,8,14,16]])[:, 0]
    phi = mdt.compute_dihedrals(traj, indices=[[4,6,8,14]])[:, 0]
    state = np.full_like(psi, False, dtype=bool)
    # phi: -pi -> 0 
    # psi: > -50 but smaller 30 degree
    deg = 180/np.pi
    state[(phi <= 0) & (-50/deg <= psi) & (psi <= 30/deg)] = True
    return state

wrapped_alphaR = arcdd.trajectory.TrajectoryFunctionWrapper(alpha_R)
wrapped_psi = arcdd.trajectory.TrajectoryFunctionWrapper(psi)
"""

from state_funcs import alpha_R, C7_eq
wrapped_alphaR = aimmdd.TrajectoryFunctionWrapper(alpha_R, call_kwargs={"scratch_dir": scratch_dir})
wrapped_C7_eq = aimmdd.TrajectoryFunctionWrapper(C7_eq, call_kwargs={"scratch_dir": scratch_dir})

In [15]:
import aimmd.distributed.logic

In [16]:
propagators = [aimmd.distributed.logic.PropagatorUntilAnyState(states=[wrapped_alphaR, wrapped_C7_eq],
                                                               engine_cls=aimmdd.GmxEngine,
                                                               engine_kwargs={"gro_file":os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                                                              "top_file":os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                                                                               "mdrun_extra_args": "-nt 4",
                                                                              },
                                                               run_config=aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp")),
                                                               walltime_per_part=0.005,
                                                               
                                                              )
               for _ in range(n_engines)
               ]

Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.


In [17]:
#initial_tp_mdt = mdt.load(os.path.join(scratch_dir, "gmx_infiles/ala_400K_TP_low_barrier.h5"))

In [18]:
#initial_tp_mdt[10].save_trr(os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"))

In [19]:
starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro"))

starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

In [20]:
wrapped_alphaR

TrajectoryFunctionWrapper(function=<function alpha_R at 0x7f995d8fd040>, call_kwargs={'scratch_dir': '/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'})

In [21]:
import mdtraj as mdt

In [22]:
mdt.load(os.path.join(scratch_dir, "gmx_infiles/conf.gro"), top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (ALA, ACE)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (NME, ALA)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (HOH, NME)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/core/trajectory.py:422: UserWarning: top= kwarg igno

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7f98b38e9fa0>

In [23]:
mdt.load(starting_conf.trajectory_file, top=starting_conf.structure_file)

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7f98b3819730>

In [24]:
alpha_R(starting_conf, **{"scratch_dir": scratch_dir})

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


array([ True])

In [25]:
await wrapped_alphaR(starting_conf)

array([ True])

In [26]:
scratch_dir

'/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'

In [27]:
tasks = [asyncio.create_task(p.propagate_and_concatenate(starting_configuration=starting_conf,
                                                         workdir=wdir,
                                                         deffnm="blib",
                                                         tra_out=os.path.join(wdir, "blub_until_state.trr"),
                                                          ))
         for p, wdir, mdp in zip(propagators, wdirs, mdps)]

done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)

In [28]:
list(done)[0]

<Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>

In [29]:
tasks

[<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() 

In [30]:
done, pending = await asyncio.wait(tasks,)# return_when=asyncio.FIRST_COMPLETED)

In [31]:
for i in done:
    print(i)
    print(tasks.index(i))

<Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
3
<Task finished name='Task-7' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
4
<Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
1
<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() g

In [32]:
pending

set()

In [34]:
start = time.time()
await asyncio.gather(*(e.prepare(starting_configuration=None, workdir=wdir, deffnm="blib", run_config=mdp)
                               for e, wdir, mdp in zip(engines, wdirs, mdps)
                               )
                            )
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.2644813060760498 s


In [35]:
walltime = 0.02 # 0.01 h = 36 s
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 72.60385823249817 s


In [36]:
trajs

[Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blib.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir0/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blib.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir1/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blib.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir2/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blib.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir3/blib.tpr),
 Trajectory(trajectory_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/engine_wdir4/blib.part0001.trr, structure_file=/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed

In [21]:
start = time.time()
result1 = await wrapped_func(trajs[-1])
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.48328495025634766 s


In [22]:
start = time.time()
results = await asyncio.gather(*(wrapped_func(t) for t in trajs))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.5739212036132812 s


In [23]:
trajs

[Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir5/blub.part0002.trr, topology_file=/home/think/scratch/arcd_dist